In [1]:
%matplotlib inline

from __future__ import absolute_import, print_function, unicode_literals, division
from sklearn.datasets import fetch_mldata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
import random as rand
import copy

In [2]:
def mat_dbg(x):
    print(x.shape, ": \n", x)

In [225]:
class MLP(object):
    """
    NOTE: Matrix operations are modified from RBM file
    In particular, we have an an an input of n data points with
    dimension d as a d-by-n matrix
    """
    def __init__(self, data, num_hidden, learn_rate):
        # First row are the target values
        self.targets = data[0]
        self.data = data[1:,:]
        self.num_data = data.shape[1]
        self.num_visible = data.shape[0] - 1
        self.num_hidden = num_hidden
        self.learn_rate = learn_rate
        
        self.min_data = np.amin(self.targets)
        self.max_data = np.amax(self.targets)
        self.state_size = 2 * (self.max_data - self.min_data) + 4
        
        self.weights = np.random.rand(self.num_visible + 1, self.num_hidden)
        
        self.out_scale = self.state_size * np.ones((self.num_hidden,1))
        self.out_weights = np.random.rand(self.num_hidden,1)
        
        self.data = np.insert(self.data, 0, 1, axis = 0)
    
    def _sigmoid(self,x):
        return 1.0/(1.0+np.exp(-x))
        
    def _activated(self,mat):
        activated = self._sigmoid(np.dot(self.weights.T,mat))
        return activated
    
    def _out_linsum(self,mat):
        linsum = np.dot(self.out_weights.T,mat)
        return linsum
    
    def _gradient(self,mat):
        w = copy.deepcopy(mat)
        for elt in np.nditer(w,op_flags=['readwrite']):
            elt[...] = elt - elt * elt
        return w
    
    def _scale(self,mat):
        scaled = np.dot(self.out_scale,mat)
        return scaled
    
    def _feed_forward(self):
        data_copy = copy.deepcopy(self.data)
        for i in range(0,self.num_data):
            data_col = data_copy[:,i]
            data_col = np.reshape(data_col, (-1, 1))
            
            activated_output = (self._activated(data_col))
            guess = self._scale(2 * activated_output - 1)
            guess = self._out_linsum(guess)
            #print("Guess: ", guess)
            
            error = self.targets[i] - guess
            grad = (self._gradient(activated_output))
            
            c = (error * self.learn_rate)[0]
            wt_change = c * grad * data_col
            
            self.weights = wt_change + self.weights
            self.weights = self.weights
        
        #print("New weights: ",self.weights)
        
    def _prop_backward(self):
        return None
    
    def _calc_error(self, mat):
        return None
    
    def pred(self, test):
        test_copy = copy.deepcopy(test)
        test_copy = np.insert(test_copy, 0, 1, axis = 0)
        pred_p = self._activated(test_copy)
        pred = 2 * pred_p - 1
        print("Pred1:", pred, " \n and size: ", np.shape(pred))
        #pred = np.dot(self.out_scale.T,pred)
        pred = 1 * np.dot(self.out_weights.T,pred)
        return pred


In [226]:
# See www.cse.unsw.edu.au/~cs9417ml/MLP2/
# https://www.hiit.fi/u/ahonkela/dippa/node41.html
dims = 3
data_pts = 50 * dims ** 2
sample_data = np.sign(2*np.random.rand(dims,data_pts)-1)
sample_data[0] = (2 * sample_data[1] + 1 * sample_data[2]) 
# Need to write binary target values for data as well for MLP
#mat_dbg(sample_data)
MLP_1 = MLP(sample_data,2,0.01)
print("Weights^T: \n",MLP_1.weights.T)
print("Targets: ")
print(MLP_1.targets)
#print("Data sample: \n",MLP_1.data[1:5])

Weights^T: 
 [[ 0.89433334  0.87292369  0.45852058]
 [ 0.29373452  0.55405624  0.36302391]]
Targets: 
[-1. -1. -1.  1. -1. -1. -1. -1.  3.  1. -1. -3.  1.  3. -3. -1. -1. -3.
 -3. -3. -3. -1.  1.  1.  1. -1. -1.  3. -3. -3.  1.  1.  3. -3. -3.  3.
 -3.  1. -1.  3.  1.  1.  3.  3. -3.  3.  1.  3.  3. -3. -1. -1. -1. -3.
 -1.  1. -1.  3. -3. -1. -3. -1. -3. -3.  3. -3. -1. -1. -1. -3.  3. -1.
 -3. -3. -1. -3.  3.  1.  3. -3. -1.  3. -1.  1. -3.  3.  3.  1.  3. -3.
 -3.  1.  3.  1. -1. -3.  3.  1.  3.  1.  3.  3.  3. -3.  1. -1. -3.  1.
 -1.  3.  1.  1. -3. -3.  1. -3.  1. -3. -1. -3.  1. -1. -1.  3. -3.  3.
  1.  3. -3. -1.  1.  3. -3. -1. -1. -3. -1.  3.  3.  1. -3.  3. -1.  1.
 -3. -1.  3.  1.  3.  1.  3. -3.  3.  3.  1. -3. -1.  1. -3.  3.  1.  1.
 -1.  1. -1.  1.  3.  3. -3.  1.  3.  1.  1.  1.  3. -3.  1. -1. -1. -1.
  3.  3.  3.  1. -3. -1. -1.  3. -1. -3.  1.  1. -1. -3.  3. -3. -3.  1.
 -1. -3.  3. -3.  3.  3. -1.  3.  3. -3. -1. -3.  3. -3.  1.  3.  3. -1.
 -1.  1. -3. -3.  1.  

In [227]:
iterations = 2
for i in range(0,iterations):
    MLP_1._feed_forward()
#MLP_1._feed_forward()
print("Weights: \n", MLP_1.weights)
results = np.dot(MLP_1.weights.T,MLP_1.data)
print("Targets: \n", MLP_1.targets)
print("Results: \n", results)

ValueError: shapes (2,1) and (2,1) not aligned: 1 (dim 1) != 2 (dim 0)

In [224]:
dims = 2
test_pts = 10
test_data = np.sign(2*np.random.rand(dims,test_pts)-1)
print(test_data)
print("targets: ", 2 * test_data[0] + 1 * test_data[1])
a = MLP_1.pred(test_data)
print("Shape:",np.shape(a))
print("Guesses: \n", a)

[[ 1. -1. -1. -1. -1. -1. -1.  1. -1.  1.]
 [-1.  1.  1. -1. -1. -1. -1. -1.  1. -1.]]
targets:  [ 1. -1. -1. -3. -3. -3. -3.  1. -1.  1.]
Pred1: [[ 0.2745526  -0.24282029 -0.24282029 -0.70129314 -0.70129314 -0.70129314
  -0.70129314  0.2745526  -0.24282029  0.2745526 ]
 [ 0.36680258 -0.134828   -0.134828   -0.71972672 -0.71972672 -0.71972672
  -0.71972672  0.36680258 -0.134828    0.36680258]]  
 and size:  (2, 10)
Shape: (1, 10)
Guesses: 
 [[ 0.08071335 -0.04311095 -0.04311095 -0.17377399 -0.17377399 -0.17377399
  -0.17377399  0.08071335 -0.04311095  0.08071335]]


In [211]:
print(MLP_1.out_weights)

[[ 0.59697608]
 [ 0.84993872]]


In [216]:
3/.2

15.0